In [1]:
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_row", 160)
pd.set_option('max_colwidth', 200)

In [2]:
a = pd.read_csv('accepted.csv')
a.head(3)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,246

In [3]:
a.shape

(2260701, 151)

In [4]:
#1. Remove all obserations with No Funded Amount First, How many:
print(len(a[a.funded_amnt.isnull()]))

33


In [5]:
drop = a[a.funded_amnt.isnull()].index
a.drop(drop, axis=0, inplace=True)
a.shape

(2260668, 151)

In [6]:
#No more NaN funded amounts: 
a[a.funded_amnt.isna()].shape

(0, 151)

In [7]:
#2. How many value types for "loan_status"?
print(a.loan_status.value_counts(dropna=False).sum()) #Total matches

a.loan_status.value_counts(dropna=False)

2260668


Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [8]:
#Create new column and map Charged Off = 0, Fully Paid = 1
a['loan_status_bin'] = a['loan_status'].map({'Charged Off':0, 'Fully Paid':1})

In [9]:
#Quick Check:
a[['loan_status', 'loan_status_bin']].sample(10)

,loan_status,loan_status_bin
103685,Fully Paid,1.0
1794469,Fully Paid,1.0
2160862,Charged Off,0.0
1936389,Fully Paid,1.0
1193673,Fully Paid,1.0
244573,Current,NaN
279993,Charged Off,0.0
1313934,Current,NaN
1918719,Fully Paid,1.0
1227932,Fully Paid,1.0


In [10]:
#Drop all loans which do not classify as "Fully Paid" or "Charged Off":
drop2 = a[a.loan_status_bin.isnull()].index
a.drop(drop2, axis=0, inplace=True)
a.shape

(1345310, 152)

In [14]:
#3. Drop member_id (all NaN), funded_amnt, funded_amnt_inv. Keep loan_amnt (we know that before investment): 
a.drop(['member_id', 'funded_amnt', 'funded_amnt_inv'], axis=1, inplace=True)
a.shape

(1345310, 149)

In [15]:
#4. Remove all columns related to "hardship" status. We won't know this at get go. 

a.drop(['hardship_flag', 'hardship_last_payment_amount', 'hardship_payoff_balance_amount',
        'hardship_loan_status', 'hardship_dpd', 'hardship_end_date', 'hardship_start_date',
        'hardship_amount', 'hardship_status', 'hardship_reason', 'hardship_type', 
        'hardship_length'], axis=1, inplace=True)
a.shape

(1345310, 137)

In [16]:
#5. Remove 'Policy Code' Column. All Policy Codes here are '1'. 
a.policy_code.value_counts()

1.0    1345310
Name: policy_code, dtype: int64

In [17]:
a.drop(['policy_code'], axis=1, inplace=True)
a.shape

(1345310, 136)

In [18]:
#6. Remove all observations with "Joint Applications". 
a.application_type.value_counts()

Individual    1319510
Joint App       25800
Name: application_type, dtype: int64

In [19]:
a = a[a.application_type == 'Individual']
a.shape

(1319510, 136)

In [20]:
#7. Remove all info regarding second applicant or "Joint Applications".

In [21]:
a.drop(['sec_app_mths_since_last_major_derog', 'sec_app_collections_12_mths_ex_med',
        'sec_app_chargeoff_within_12_mths', 'sec_app_num_rev_accts', 'sec_app_open_act_il',
        'sec_app_revol_util', 'sec_app_revol_util', 'sec_app_open_acc', 'sec_app_mort_acc',
        'sec_app_inq_last_6mths', 'sec_app_earliest_cr_line', 'sec_app_fico_range_low',
        'dti_joint', 'sec_app_fico_range_high', 'verification_status_joint', 'revol_bal_joint', 
        'annual_inc_joint'], axis=1, inplace=True)

a.shape

(1319510, 120)

In [22]:
#8. Remove 'next_pymnt_d' because all loans are "Paid Off" or "Defaulted", so they are all "NaN"
a.next_pymnt_d.isna().sum()

1319510

In [23]:
a.drop(['next_pymnt_d'], axis=1, inplace=True)
a.shape

(1319510, 119)

In [24]:
#9. Drop 'url'. It will not provide any info. 
a.drop(['url'], axis=1, inplace=True)
a.shape

(1319510, 118)

In [25]:
#10. Drop 'zip_code'. It's not even complete. Can always use addr_state if location is a factor. 
a.drop(['zip_code'], axis=1, inplace=True)
a.shape

(1319510, 117)

In [26]:
#11. "out_prncp" and "out_prncp_inv" are 0 because there is no more "outstanding" amounts. Drop them. 
print(a.out_prncp.value_counts())
print(a.out_prncp_inv.value_counts())

0.0    1319510
Name: out_prncp, dtype: int64
0.0    1319510
Name: out_prncp_inv, dtype: int64


In [27]:
a.drop(['out_prncp', 'out_prncp_inv'], axis=1, inplace=True)
a.shape

(1319510, 115)

In [28]:
#12. Remove 'id' (irrevelvant) but keep 'issue_d' for feature engineering.
a.drop(['id'], axis=1, inplace=True)
a.shape

(1319510, 114)

In [29]:
#13. Drop 'pymnt_plan'. They are all 'n'. 
a.pymnt_plan.value_counts()

n    1319510
Name: pymnt_plan, dtype: int64

In [30]:
a.drop(['pymnt_plan'], axis=1, inplace=True)
a.shape

(1319510, 113)

In [31]:
#14. Drop 'last_fico_range_high' and 'last_fico_range_low'. We only rely on initial fico scores. 
a.drop(['last_fico_range_high', 'last_fico_range_low'], axis=1, inplace=True)
a.shape

(1319510, 111)

In [49]:
#Partial Set of Features, Work in Progress......

In [32]:
#15. Let's rearrange order of columns to enhance our view: 
sub = a[['issue_d', 'loan_amnt', 'annual_inc', 'dti', 'fico_range_low', 'fico_range_high',
   'term', 'int_rate', 'installment', 'initial_list_status',
   'grade', 'sub_grade', 'emp_length', 'home_ownership',
   'earliest_cr_line', 'open_acc', 'total_acc',
   'revol_bal', 'revol_util',
   'inq_last_6mths',
   
   'delinq_2yrs', 'mths_since_last_delinq','acc_now_delinq',
   'collections_12_mths_ex_med','tot_coll_amt', 'tot_cur_bal',
   'pub_rec', 'mths_since_last_record',
   
   
   'purpose', 'title', 'addr_state', 'loan_status', 
   'loan_status_bin', 
   'application_type',
   'verification_status','emp_title',
   'total_pymnt', 'total_pymnt_inv',
   'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
   'recoveries', 'collection_recovery_fee',
   'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 
   'desc']]

sub.head(2)

,issue_d,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,earliest_cr_line,open_acc,total_acc,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,pub_rec,mths_since_last_record,purpose,title,addr_state,loan_status,loan_status_bin,application_type,verification_status,emp_title,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,desc
0,Dec-2015,3600.0,55000.0,5.91,675.0,679.0,36 months,13.99,123.03,w,C,C4,10+ years,MORTGAGE,Aug-2003,7.0,13.0,2765.0,29.7,1.0,0.0,30.0,0.0,0.0,722.0,144904.0,0.0,NaN,debt_consolidation,Debt consolidation,PA,Fully Paid,1.0,Individual,Not Verified,leadman,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,Mar-2019,NaN
1,Dec-2015,24700.0,65000.0,16.06,715.0,719.0,36 months,11.99,820.28,w,C,C1,10+ years,MORTGAGE,Dec-1999,22.0,38.0,21470.0,19.2,4.0,1.0,6.0,0.0,0.0,0.0,204396.0,0.0,NaN,small_business,Business,SD,Fully Paid,1.0,Individual,Not Verified,Engineer,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,Mar-2019,NaN


In [33]:
sub.shape

(1319510, 47)

In [34]:
sub.isna().mean().sort_values()

issue_d                       0.000000e+00
application_type              0.000000e+00
loan_status_bin               0.000000e+00
loan_status                   0.000000e+00
addr_state                    0.000000e+00
total_pymnt_inv               0.000000e+00
purpose                       0.000000e+00
total_rec_prncp               0.000000e+00
pub_rec                       0.000000e+00
total_rec_int                 0.000000e+00
total_rec_late_fee            0.000000e+00
acc_now_delinq                0.000000e+00
recoveries                    0.000000e+00
delinq_2yrs                   0.000000e+00
collection_recovery_fee       0.000000e+00
verification_status           0.000000e+00
revol_bal                     0.000000e+00
open_acc                      0.000000e+00
loan_amnt                     0.000000e+00
annual_inc                    0.000000e+00
dti                           0.000000e+00
fico_range_low                0.000000e+00
fico_range_high               0.000000e+00
term       

In [35]:
#Carve out features from 'sub' which you think are most important:

imp_sub = sub[['issue_d', 'loan_amnt', 'annual_inc', 'dti', 'fico_range_low', 'fico_range_high',
   'term', 'int_rate', 'installment', 'initial_list_status',
   'grade', 'sub_grade', 'emp_length', 'home_ownership',
   'earliest_cr_line', 'open_acc', 'total_acc',
   'revol_bal', 'revol_util',
   'inq_last_6mths', 'verification_status',
   
   'delinq_2yrs', 'mths_since_last_delinq','acc_now_delinq',
   'collections_12_mths_ex_med','tot_coll_amt', 'tot_cur_bal',
   'pub_rec', 'mths_since_last_record',
   
   
   'purpose', 'title', 'loan_status', 
   'loan_status_bin']] 

In [36]:
imp_sub.shape

(1319510, 33)

In [37]:
imp_sub.head(2)

,issue_d,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,earliest_cr_line,open_acc,total_acc,revol_bal,revol_util,inq_last_6mths,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,pub_rec,mths_since_last_record,purpose,title,loan_status,loan_status_bin
0,Dec-2015,3600.0,55000.0,5.91,675.0,679.0,36 months,13.99,123.03,w,C,C4,10+ years,MORTGAGE,Aug-2003,7.0,13.0,2765.0,29.7,1.0,Not Verified,0.0,30.0,0.0,0.0,722.0,144904.0,0.0,NaN,debt_consolidation,Debt consolidation,Fully Paid,1.0
1,Dec-2015,24700.0,65000.0,16.06,715.0,719.0,36 months,11.99,820.28,w,C,C1,10+ years,MORTGAGE,Dec-1999,22.0,38.0,21470.0,19.2,4.0,Not Verified,1.0,6.0,0.0,0.0,0.0,204396.0,0.0,NaN,small_business,Business,Fully Paid,1.0


In [38]:
imp_sub.isna().sum().sort_values()

issue_d                             0
purpose                             0
pub_rec                             0
acc_now_delinq                      0
delinq_2yrs                         0
verification_status                 0
revol_bal                           0
loan_status                         0
open_acc                            0
earliest_cr_line                    0
home_ownership                      0
total_acc                           0
loan_status_bin                     0
annual_inc                          0
dti                                 0
sub_grade                           0
fico_range_low                      0
fico_range_high                     0
int_rate                            0
installment                         0
initial_list_status                 0
grade                               0
term                                0
loan_amnt                           0
inq_last_6mths                      1
collections_12_mths_ex_med         56
revol_util  

In [39]:
#Let's add emp_length to the mix. Impute NaN with "None" string for time being: 

In [40]:
imp_sub['emp_length'].isna().sum()

74764

In [41]:
imp_sub['emp_length'].replace(np.nan, "None", inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [42]:
imp_sub['emp_length'].isna().sum()

0

In [43]:
imp_sub.isna().sum().sort_values()

issue_d                             0
purpose                             0
pub_rec                             0
acc_now_delinq                      0
delinq_2yrs                         0
verification_status                 0
revol_bal                           0
loan_status                         0
open_acc                            0
earliest_cr_line                    0
home_ownership                      0
emp_length                          0
total_acc                           0
grade                               0
loan_amnt                           0
annual_inc                          0
dti                                 0
sub_grade                           0
fico_range_low                      0
fico_range_high                     0
loan_status_bin                     0
term                                0
int_rate                            0
installment                         0
initial_list_status                 0
inq_last_6mths                      1
collections_

In [44]:
#Let's take a closer look at "mths_since_last_delinq":
imp_sub.mths_since_last_delinq.value_counts(dropna=False)

NaN      665316
12.0      12523
13.0      12477
9.0       12434
6.0       12385
          ...  
192.0         1
137.0         1
226.0         1
178.0         1
202.0         1
Name: mths_since_last_delinq, Length: 164, dtype: int64

In [45]:
imp_sub[imp_sub.mths_since_last_delinq.isna()][['loan_status_bin']].describe()

,loan_status_bin
count,665316.000000
mean,0.808199
std,0.393717
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [46]:
#Because 80% of all NaN values for "last_delinq" have been paid off, lets impute with "0". 
imp_sub['mths_since_last_delinq'] = imp_sub['mths_since_last_delinq'].replace(np.nan, 0)

/var/folders/kl/4xt5tydd0xb8yvwkgnx2mnc00000gn/T/ipykernel_26204/762310108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imp_sub['mths_since_last_delinq'] = imp_sub['mths_since_last_delinq'].replace(np.nan, 0)


In [47]:
imp_sub.isna().sum().sort_values()

issue_d                             0
purpose                             0
pub_rec                             0
acc_now_delinq                      0
mths_since_last_delinq              0
delinq_2yrs                         0
verification_status                 0
revol_bal                           0
loan_status                         0
open_acc                            0
earliest_cr_line                    0
home_ownership                      0
total_acc                           0
sub_grade                           0
loan_amnt                           0
annual_inc                          0
dti                                 0
emp_length                          0
fico_range_high                     0
term                                0
fico_range_low                      0
int_rate                            0
installment                         0
initial_list_status                 0
grade                               0
loan_status_bin                     0
inq_last_6mt

In [48]:
#For the time being, drop all other features with NAs....we don't know which ones are important.
#We also don't know what is the best way to "impute".
No_NA = imp_sub.dropna(axis=1)
No_NA

,issue_d,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,earliest_cr_line,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin
0,Dec-2015,3600.0,55000.0,5.91,675.0,679.0,36 months,13.99,123.03,w,C,C4,10+ years,MORTGAGE,Aug-2003,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0
1,Dec-2015,24700.0,65000.0,16.06,715.0,719.0,36 months,11.99,820.28,w,C,C1,10+ years,MORTGAGE,Dec-1999,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0
4,Dec-2015,10400.0,104433.0,25.37,695.0,699.0,60 months,22.45,289.91,w,F,F1,3 years,MORTGAGE,Jun-1998,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0
5,Dec-2015,11950.0,34000.0,10.20,690.0,694.0,36 months,13.44,405.18,w,C,C3,4 years,RENT,Oct-1987,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0
6,Dec-2015,20000.0,180000.0,14.67,680.0,684.0,36 months,9.17,637.58,f,B,B2,10+ years,MORTGAGE,Jun-1990,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,debt_consolidation,Fully Paid,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,Oct-2016,18000.0,130000.0,20.59,735.0,739.0,60 months,9.49,377.95,f,B,B2,5 years,OWN,Jul-2004,17.0,39.0,23833.0,Not Verified,0.0,0.0,0.0,0.0,home_improvement,Fully Paid,1.0
2260690,Oct-2016,29400.0,180792.0,22.03,705.0,709.0,60 months,13.99,683.94,f,C,C3,9 years,MORTGAGE,Mar-2002,16.0,32.0,77480.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0
2260691,Oct-2016,32000.0,157000.0,10.34,735.0,739.0,60 months,14.49,752.74,f,C,C4,3 years,MORTGAGE,Jun-2011,14.0,18.0,111598.0,Source Verified,0.0,0.0,0.0,0.0,home_improvement,Charged Off,0.0
2260692,Oct-2016,16000.0,150000.0,12.25,665.0,669.0,60 months,12.79,362.34,f,C,C1,10+ years,RENT,Aug-1997,12.0,28.0,7700.0,Not Verified,0.0,68.0,0.0,4.0,medical,Fully Paid,1.0


In [49]:
#We now have 31 features (includes target as well)
No_NA.shape

(1319510, 26)

In [51]:
#Cross check that dataframe in use has no more Null values:
No_NA.isna().sum().sum()

0

In [52]:
data = No_NA.copy()

In [53]:
#Data Types of Remaining Features: 
data.dtypes

issue_d                    object
loan_amnt                 float64
annual_inc                float64
dti                       float64
fico_range_low            float64
fico_range_high           float64
term                       object
int_rate                  float64
installment               float64
initial_list_status        object
grade                      object
sub_grade                  object
emp_length                 object
home_ownership             object
earliest_cr_line           object
open_acc                  float64
total_acc                 float64
revol_bal                 float64
verification_status        object
delinq_2yrs               float64
mths_since_last_delinq    float64
acc_now_delinq            float64
pub_rec                   float64
purpose                    object
loan_status                object
loan_status_bin           float64
dtype: object

In [54]:
#Return unique values of categorical columns:
{column: list(data[column].unique()) for column in data.columns if data.dtypes[column]=='object'}

{'issue_d': ['Dec-2015',
  'Nov-2015',
  'Oct-2015',
  'Sep-2015',
  'Aug-2015',
  'Jul-2015',
  'Jun-2015',
  'May-2015',
  'Apr-2015',
  'Mar-2015',
  'Feb-2015',
  'Jan-2015',
  'Mar-2018',
  'Feb-2018',
  'Jan-2018',
  'Sep-2017',
  'Aug-2017',
  'Jul-2017',
  'Jun-2016',
  'May-2016',
  'Apr-2016',
  'Sep-2018',
  'Aug-2018',
  'Jul-2018',
  'Jun-2017',
  'May-2017',
  'Apr-2017',
  'Mar-2016',
  'Feb-2016',
  'Jan-2016',
  'Dec-2014',
  'Nov-2014',
  'Oct-2014',
  'Sep-2014',
  'Aug-2014',
  'Jul-2014',
  'Jun-2014',
  'May-2014',
  'Apr-2014',
  'Mar-2014',
  'Feb-2014',
  'Jan-2014',
  'Dec-2018',
  'Nov-2018',
  'Oct-2018',
  'Jun-2018',
  'May-2018',
  'Apr-2018',
  'Dec-2011',
  'Nov-2011',
  'Oct-2011',
  'Sep-2011',
  'Aug-2011',
  'Jul-2011',
  'Jun-2011',
  'May-2011',
  'Apr-2011',
  'Mar-2011',
  'Feb-2011',
  'Jan-2011',
  'Dec-2010',
  'Nov-2010',
  'Oct-2010',
  'Sep-2010',
  'Aug-2010',
  'Jul-2010',
  'Jun-2010',
  'May-2010',
  'Apr-2010',
  'Mar-2010',
  'Feb-20

In [55]:
#A. Working with Date Columns First: 

In [56]:
date_columns = ['issue_d', 'earliest_cr_line']

In [57]:
data[date_columns]

,issue_d,earliest_cr_line
0,Dec-2015,Aug-2003
1,Dec-2015,Dec-1999
4,Dec-2015,Jun-1998
5,Dec-2015,Oct-1987
6,Dec-2015,Jun-1990
...,...,...
2260688,Oct-2016,Jul-2004
2260690,Oct-2016,Mar-2002
2260691,Oct-2016,Jun-2011
2260692,Oct-2016,Aug-1997


In [58]:
data.loc[0, 'earliest_cr_line'][0:3]

'Aug'

In [59]:
data.loc[0, 'earliest_cr_line'][-4:]

'2003'

In [60]:
for column in date_columns:
    data[column + '_month'] = data[column].apply(lambda x: x[0:3])
    data[column + '_year'] = data[column].apply(lambda x: x[-4:])

In [61]:
data

,issue_d,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,earliest_cr_line,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year
0,Dec-2015,3600.0,55000.0,5.91,675.0,679.0,36 months,13.99,123.03,w,C,C4,10+ years,MORTGAGE,Aug-2003,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,Dec,2015,Aug,2003
1,Dec-2015,24700.0,65000.0,16.06,715.0,719.0,36 months,11.99,820.28,w,C,C1,10+ years,MORTGAGE,Dec-1999,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,Dec,2015,Dec,1999
4,Dec-2015,10400.0,104433.0,25.37,695.0,699.0,60 months,22.45,289.91,w,F,F1,3 years,MORTGAGE,Jun-1998,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0,Dec,2015,Jun,1998
5,Dec-2015,11950.0,34000.0,10.20,690.0,694.0,36 months,13.44,405.18,w,C,C3,4 years,RENT,Oct-1987,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,Dec,2015,Oct,1987
6,Dec-2015,20000.0,180000.0,14.67,680.0,684.0,36 months,9.17,637.58,f,B,B2,10+ years,MORTGAGE,Jun-1990,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,Dec,2015,Jun,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,Oct-2016,18000.0,130000.0,20.59,735.0,739.0,60 months,9.49,377.95,f,B,B2,5 years,OWN,Jul-2004,17.0,39.0,23833.0,Not Verified,0.0,0.0,0.0,0.0,home_improvement,Fully Paid,1.0,Oct,2016,Jul,2004
2260690,Oct-2016,29400.0,180792.0,22.03,705.0,709.0,60 months,13.99,683.94,f,C,C3,9 years,MORTGAGE,Mar-2002,16.0,32.0,77480.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,Oct,2016,Mar,2002
2260691,Oct-2016,32000.0,157000.0,10.34,735.0,739.0,60 months,14.49,752.74,f,C,C4,3 years,MORTGAGE,Jun-2011,14.0,18.0,111598.0,Source Verified,0.0,0.0,0.0,0.0,home_improvement,Charged Off,0.0,Oct,2016,Jun,2011
2260692,Oct-2016,16000.0,150000.0,12.25,665.0,669.0,60 months,12.79,362.34,f,C,C1,10+ years,RENT,Aug-1997,12.0,28.0,7700.0,Not Verified,0.0,68.0,0.0,4.0,medical,Fully Paid,1.0,Oct,2016,Aug,1997


In [62]:
data = data.drop(date_columns, axis=1)
data.shape

(1319510, 28)

In [63]:
month_ordering = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [64]:
for column in date_columns: 
    data[column + '_month'] = data[column + '_month'].apply(lambda x: month_ordering.index(x))

In [65]:
data

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year
0,3600.0,55000.0,5.91,675.0,679.0,36 months,13.99,123.03,w,C,C4,10+ years,MORTGAGE,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,11,2015,7,2003
1,24700.0,65000.0,16.06,715.0,719.0,36 months,11.99,820.28,w,C,C1,10+ years,MORTGAGE,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,11,2015,11,1999
4,10400.0,104433.0,25.37,695.0,699.0,60 months,22.45,289.91,w,F,F1,3 years,MORTGAGE,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0,11,2015,5,1998
5,11950.0,34000.0,10.20,690.0,694.0,36 months,13.44,405.18,w,C,C3,4 years,RENT,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,11,2015,9,1987
6,20000.0,180000.0,14.67,680.0,684.0,36 months,9.17,637.58,f,B,B2,10+ years,MORTGAGE,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,11,2015,5,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,18000.0,130000.0,20.59,735.0,739.0,60 months,9.49,377.95,f,B,B2,5 years,OWN,17.0,39.0,23833.0,Not Verified,0.0,0.0,0.0,0.0,home_improvement,Fully Paid,1.0,9,2016,6,2004
2260690,29400.0,180792.0,22.03,705.0,709.0,60 months,13.99,683.94,f,C,C3,9 years,MORTGAGE,16.0,32.0,77480.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,9,2016,2,2002
2260691,32000.0,157000.0,10.34,735.0,739.0,60 months,14.49,752.74,f,C,C4,3 years,MORTGAGE,14.0,18.0,111598.0,Source Verified,0.0,0.0,0.0,0.0,home_improvement,Charged Off,0.0,9,2016,5,2011
2260692,16000.0,150000.0,12.25,665.0,669.0,60 months,12.79,362.34,f,C,C1,10+ years,RENT,12.0,28.0,7700.0,Not Verified,0.0,68.0,0.0,4.0,medical,Fully Paid,1.0,9,2016,7,1997


In [66]:
for column in data.columns:
    try:
        data[column] = data[column].astype(np.float)
    except:
        pass

In [67]:
#Create new column of total months borrower has credit line upto loan issuance date:
data = data.assign(months_cr_line=(data['issue_d_year'] - data['earliest_cr_line_year'])*12+(data['issue_d_month']-data['earliest_cr_line_month']))            

In [68]:
data.drop(['issue_d_month', 'issue_d_year', 'earliest_cr_line_month', 'earliest_cr_line_year'], axis=1, inplace=True)
data.shape

(1319510, 25)

In [69]:
data.head(2)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,months_cr_line
0,3600.0,55000.0,5.91,675.0,679.0,36 months,13.99,123.03,w,C,C4,10+ years,MORTGAGE,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,148.0
1,24700.0,65000.0,16.06,715.0,719.0,36 months,11.99,820.28,w,C,C1,10+ years,MORTGAGE,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,192.0


In [70]:
data.dtypes

loan_amnt                 float64
annual_inc                float64
dti                       float64
fico_range_low            float64
fico_range_high           float64
term                       object
int_rate                  float64
installment               float64
initial_list_status        object
grade                      object
sub_grade                  object
emp_length                 object
home_ownership             object
open_acc                  float64
total_acc                 float64
revol_bal                 float64
verification_status        object
delinq_2yrs               float64
mths_since_last_delinq    float64
acc_now_delinq            float64
pub_rec                   float64
purpose                    object
loan_status                object
loan_status_bin           float64
months_cr_line            float64
dtype: object

In [71]:
#Return unique values of categorical columns:
{column: list(data[column].unique()) for column in data.columns if data.dtypes[column]=='object'}

{'term': [' 36 months', ' 60 months'],
 'initial_list_status': ['w', 'f'],
 'grade': ['C', 'F', 'B', 'A', 'E', 'D', 'G'],
 'sub_grade': ['C4',
  'C1',
  'F1',
  'C3',
  'B2',
  'B1',
  'A2',
  'B5',
  'C2',
  'E2',
  'A4',
  'E3',
  'C5',
  'A1',
  'D4',
  'F3',
  'B4',
  'D1',
  'B3',
  'D3',
  'D5',
  'A5',
  'F2',
  'E4',
  'D2',
  'E1',
  'F5',
  'E5',
  'A3',
  'G2',
  'G1',
  'G3',
  'G4',
  'F4',
  'G5'],
 'emp_length': ['10+ years',
  '3 years',
  '4 years',
  '6 years',
  '7 years',
  '8 years',
  '2 years',
  '5 years',
  '9 years',
  '< 1 year',
  '1 year',
  'None'],
 'home_ownership': ['MORTGAGE', 'RENT', 'OWN', 'ANY', 'NONE', 'OTHER'],
 'verification_status': ['Not Verified', 'Source Verified', 'Verified'],
 'purpose': ['debt_consolidation',
  'small_business',
  'major_purchase',
  'credit_card',
  'other',
  'home_improvement',
  'house',
  'vacation',
  'car',
  'medical',
  'moving',
  'renewable_energy',
  'wedding',
  'educational'],
 'loan_status': ['Fully Paid', '

In [72]:
#Encoding Functions

def binary_temp(df, column, positive_value):
    temp_df = df.assign(new_be=df[column].apply(lambda x: 1 if x == positive_value else 0))
    return temp_df

def binary_encode(temp_df, column):
    new_df = temp_df.drop(column, axis=1)
    new_df = new_df.rename(columns={'new_be':column})
    return new_df
    
    #df[column + '_be'] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    #df_new = pd.concat([df, df[column].apply(lambda x: 1 if x == positive_value else 0)], axis=1)
    #df_new = df_new.drop(column, axis=1)
    #return df_new

def ordinal_temp(df, column, ordering):
    temp_df = df.assign(new_oe=df[column].apply(lambda x: ordering.index(x)))
    return temp_df

def ordinal_encode(temp_df, column):
    new_df = temp_df.drop(column, axis=1)
    new_df = new_df.rename(columns={'new_oe':column})
    return new_df

def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column])
    df_new = pd.concat([df, dummies], axis=1)
    df_new = df_new.drop(column, axis=1)
    return df_new

In [73]:
#Check Binary Encoding:
temp = binary_temp(data, 'term', ' 60 months')

In [74]:
temp[['term', 'new_be']].sample(10)

,term,new_be
2035076,36 months,0
1913194,36 months,0
303600,60 months,1
1302464,36 months,0
2018786,60 months,1
2028070,36 months,0
878715,36 months,0
1128874,36 months,0
408314,36 months,0
1260704,36 months,0


In [75]:
#If ok, complete binary encoding:
new_df = binary_encode(temp, 'term')
new_df.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,months_cr_line,term
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,w,C,C4,10+ years,MORTGAGE,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,148.0,0
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,w,C,C1,10+ years,MORTGAGE,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,192.0,0
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,w,F,F1,3 years,MORTGAGE,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0,210.0,1
5,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,w,C,C3,4 years,RENT,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,338.0,0
6,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,f,B,B2,10+ years,MORTGAGE,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,306.0,0


In [76]:
temp = binary_temp(new_df, 'initial_list_status', 'w')

In [77]:
temp[['initial_list_status', 'new_be']].sample(10)

,initial_list_status,new_be
1025600,w,1
1715045,f,0
1159515,f,0
1227589,w,1
620441,f,0
263596,f,0
1736527,f,0
1149185,w,1
416508,f,0
1275935,f,0


In [78]:
new_df = binary_encode(temp, 'initial_list_status')
new_df.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,grade,sub_grade,emp_length,home_ownership,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,months_cr_line,term,initial_list_status
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,C,C4,10+ years,MORTGAGE,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,148.0,0,1
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,C,C1,10+ years,MORTGAGE,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,192.0,0,1
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,F,F1,3 years,MORTGAGE,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0,210.0,1,1
5,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,C,C3,4 years,RENT,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,338.0,0,1
6,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,B,B2,10+ years,MORTGAGE,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,306.0,0,0


In [79]:
grade_ordering = sorted(new_df['grade'].unique())
grade_ordering

['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [80]:
temp_df = ordinal_temp(new_df, 'grade', grade_ordering)
temp_df[['grade', 'new_oe']]

,grade,new_oe
0,C,2
1,C,2
4,F,5
5,C,2
6,B,1
...,...,...
2260688,B,1
2260690,C,2
2260691,C,2
2260692,C,2


In [81]:
#If ok, complete ordinal encoding:
new_df = ordinal_encode(temp_df, 'grade')
new_df.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,sub_grade,emp_length,home_ownership,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,months_cr_line,term,initial_list_status,grade
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,C4,10+ years,MORTGAGE,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,148.0,0,1,2
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,C1,10+ years,MORTGAGE,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,192.0,0,1,2
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,F1,3 years,MORTGAGE,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0,210.0,1,1,5
5,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,C3,4 years,RENT,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,338.0,0,1,2
6,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,B2,10+ years,MORTGAGE,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,306.0,0,0,1


In [82]:
sub_grade_ordering = sorted(new_df['sub_grade'].unique())
sub_grade_ordering

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'E1',
 'E2',
 'E3',
 'E4',
 'E5',
 'F1',
 'F2',
 'F3',
 'F4',
 'F5',
 'G1',
 'G2',
 'G3',
 'G4',
 'G5']

In [83]:
temp_df = ordinal_temp(new_df, 'sub_grade', sub_grade_ordering)
temp_df[['sub_grade', 'new_oe']]

,sub_grade,new_oe
0,C4,13
1,C1,10
4,F1,25
5,C3,12
6,B2,6
...,...,...
2260688,B2,6
2260690,C3,12
2260691,C4,13
2260692,C1,10


In [84]:
#If ok, complete ordinal encoding:
new_df = ordinal_encode(temp_df, 'sub_grade')
new_df.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,emp_length,home_ownership,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,months_cr_line,term,initial_list_status,grade,sub_grade
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,10+ years,MORTGAGE,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,148.0,0,1,2,13
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,10+ years,MORTGAGE,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,192.0,0,1,2,10
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,3 years,MORTGAGE,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0,210.0,1,1,5,25
5,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,4 years,RENT,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,338.0,0,1,2,12
6,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,10+ years,MORTGAGE,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,306.0,0,0,1,6


In [85]:
emp_ordering = [
    'None',
    '< 1 year',
    '1 year',
    '2 years',
    '3 years',
    '4 years',
    '5 years',
    '6 years',
    '7 years',
    '8 years',
    '9 years',
    '10+ years',
    ]

In [86]:
temp_df = ordinal_temp(new_df, 'emp_length', emp_ordering)
temp_df[['emp_length', 'new_oe']]

,emp_length,new_oe
0,10+ years,11
1,10+ years,11
4,3 years,4
5,4 years,5
6,10+ years,11
...,...,...
2260688,5 years,6
2260690,9 years,10
2260691,3 years,4
2260692,10+ years,11


In [87]:
new_df = ordinal_encode(temp_df, 'emp_length')
new_df.head(3)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,home_ownership,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,months_cr_line,term,initial_list_status,grade,sub_grade,emp_length
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,MORTGAGE,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,148.0,0,1,2,13,11
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,MORTGAGE,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,192.0,0,1,2,10,11
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,MORTGAGE,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0,210.0,1,1,5,25,4


In [88]:
#Dummify nominal features:
numerical_df = onehot_encode(new_df, 'home_ownership')
numerical_df

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,purpose,loan_status,loan_status_bin,months_cr_line,term,initial_list_status,grade,sub_grade,emp_length,ANY,MORTGAGE,NONE,OTHER,OWN,RENT
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,148.0,0,1,2,13,11,0,1,0,0,0,0
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,small_business,Fully Paid,1.0,192.0,0,1,2,10,11,0,1,0,0,0,0
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,major_purchase,Fully Paid,1.0,210.0,1,1,5,25,4,0,1,0,0,0,0
5,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,338.0,0,1,2,12,5,0,0,0,0,0,1
6,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,306.0,0,0,1,6,11,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,18000.0,130000.0,20.59,735.0,739.0,9.49,377.95,17.0,39.0,23833.0,Not Verified,0.0,0.0,0.0,0.0,home_improvement,Fully Paid,1.0,147.0,1,0,1,6,6,0,0,0,0,1,0
2260690,29400.0,180792.0,22.03,705.0,709.0,13.99,683.94,16.0,32.0,77480.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,175.0,1,0,2,12,10,0,1,0,0,0,0
2260691,32000.0,157000.0,10.34,735.0,739.0,14.49,752.74,14.0,18.0,111598.0,Source Verified,0.0,0.0,0.0,0.0,home_improvement,Charged Off,0.0,64.0,1,0,2,13,4,0,1,0,0,0,0
2260692,16000.0,150000.0,12.25,665.0,669.0,12.79,362.34,12.0,28.0,7700.0,Not Verified,0.0,68.0,0.0,4.0,medical,Fully Paid,1.0,230.0,1,0,2,10,11,0,0,0,0,0,1


In [89]:
numerical_df2 = onehot_encode(numerical_df, 'purpose')
numerical_df2

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,loan_status,loan_status_bin,months_cr_line,term,initial_list_status,grade,sub_grade,emp_length,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,7.0,13.0,2765.0,Not Verified,0.0,30.0,0.0,0.0,Fully Paid,1.0,148.0,0,1,2,13,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,22.0,38.0,21470.0,Not Verified,1.0,6.0,0.0,0.0,Fully Paid,1.0,192.0,0,1,2,10,11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,Source Verified,1.0,12.0,0.0,0.0,Fully Paid,1.0,210.0,1,1,5,25,4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,5.0,6.0,8822.0,Source Verified,0.0,0.0,0.0,0.0,Fully Paid,1.0,338.0,0,1,2,12,5,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,12.0,27.0,87329.0,Not Verified,0.0,49.0,0.0,0.0,Fully Paid,1.0,306.0,0,0,1,6,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,18000.0,130000.0,20.59,735.0,739.0,9.49,377.95,17.0,39.0,23833.0,Not Verified,0.0,0.0,0.0,0.0,Fully Paid,1.0,147.0,1,0,1,6,6,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2260690,29400.0,180792.0,22.03,705.0,709.0,13.99,683.94,16.0,32.0,77480.0,Not Verified,0.0,0.0,0.0,0.0,Fully Paid,1.0,175.0,1,0,2,12,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2260691,32000.0,157000.0,10.34,735.0,739.0,14.49,752.74,14.0,18.0,111598.0,Source Verified,0.0,0.0,0.0,0.0,Charged Off,0.0,64.0,1,0,2,13,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2260692,16000.0,150000.0,12.25,665.0,669.0,12.79,362.34,12.0,28.0,7700.0,Not Verified,0.0,68.0,0.0,4.0,Fully Paid,1.0,230.0,1,0,2,10,11,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [90]:
numerical_df3 = onehot_encode(numerical_df2, 'verification_status')
numerical_df3

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,loan_status,loan_status_bin,months_cr_line,term,initial_list_status,grade,sub_grade,emp_length,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,7.0,13.0,2765.0,0.0,30.0,0.0,0.0,Fully Paid,1.0,148.0,0,1,2,13,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,22.0,38.0,21470.0,1.0,6.0,0.0,0.0,Fully Paid,1.0,192.0,0,1,2,10,11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,1.0,12.0,0.0,0.0,Fully Paid,1.0,210.0,1,1,5,25,4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
5,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,5.0,6.0,8822.0,0.0,0.0,0.0,0.0,Fully Paid,1.0,338.0,0,1,2,12,5,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,12.0,27.0,87329.0,0.0,49.0,0.0,0.0,Fully Paid,1.0,306.0,0,0,1,6,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,18000.0,130000.0,20.59,735.0,739.0,9.49,377.95,17.0,39.0,23833.0,0.0,0.0,0.0,0.0,Fully Paid,1.0,147.0,1,0,1,6,6,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2260690,29400.0,180792.0,22.03,705.0,709.0,13.99,683.94,16.0,32.0,77480.0,0.0,0.0,0.0,0.0,Fully Paid,1.0,175.0,1,0,2,12,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2260691,32000.0,157000.0,10.34,735.0,739.0,14.49,752.74,14.0,18.0,111598.0,0.0,0.0,0.0,0.0,Charged Off,0.0,64.0,1,0,2,13,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2260692,16000.0,150000.0,12.25,665.0,669.0,12.79,362.34,12.0,28.0,7700.0,0.0,68.0,0.0,4.0,Fully Paid,1.0,230.0,1,0,2,10,11,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [106]:
numerical_df4 = numerical_df3.drop('loan_status', axis=1)
numerical_df4

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,loan_status_bin,months_cr_line,term,initial_list_status,grade,sub_grade,emp_length,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,7.0,13.0,2765.0,0.0,30.0,0.0,0.0,1.0,148.0,0,1,2,13,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,22.0,38.0,21470.0,1.0,6.0,0.0,0.0,1.0,192.0,0,1,2,10,11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,1.0,12.0,0.0,0.0,1.0,210.0,1,1,5,25,4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
5,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,5.0,6.0,8822.0,0.0,0.0,0.0,0.0,1.0,338.0,0,1,2,12,5,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,12.0,27.0,87329.0,0.0,49.0,0.0,0.0,1.0,306.0,0,0,1,6,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260688,18000.0,130000.0,20.59,735.0,739.0,9.49,377.95,17.0,39.0,23833.0,0.0,0.0,0.0,0.0,1.0,147.0,1,0,1,6,6,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2260690,29400.0,180792.0,22.03,705.0,709.0,13.99,683.94,16.0,32.0,77480.0,0.0,0.0,0.0,0.0,1.0,175.0,1,0,2,12,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2260691,32000.0,157000.0,10.34,735.0,739.0,14.49,752.74,14.0,18.0,111598.0,0.0,0.0,0.0,0.0,0.0,64.0,1,0,2,13,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2260692,16000.0,150000.0,12.25,665.0,669.0,12.79,362.34,12.0,28.0,7700.0,0.0,68.0,0.0,4.0,1.0,230.0,1,0,2,10,11,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [107]:
numerical_df4.dtypes

loan_amnt                 float64
annual_inc                float64
dti                       float64
fico_range_low            float64
fico_range_high           float64
int_rate                  float64
installment               float64
open_acc                  float64
total_acc                 float64
revol_bal                 float64
delinq_2yrs               float64
mths_since_last_delinq    float64
acc_now_delinq            float64
pub_rec                   float64
loan_status_bin           float64
months_cr_line            float64
term                        int64
initial_list_status         int64
grade                       int64
sub_grade                   int64
emp_length                  int64
ANY                         uint8
MORTGAGE                    uint8
NONE                        uint8
OTHER                       uint8
OWN                         uint8
RENT                        uint8
car                         uint8
credit_card                 uint8
debt_consolida

In [108]:
#Don't Drop the Grade Column!! they are all types now!

#numerical_df4 = numerical_df4.drop('grade', axis=1)

In [109]:
numerical_df4.shape

(1319510, 44)

In [110]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [111]:
X = numerical_df4.drop('loan_status_bin', axis=1)
y = numerical_df4['loan_status_bin']

In [112]:
X.reset_index(inplace=True, drop=True)
X

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,months_cr_line,term,initial_list_status,grade,sub_grade,emp_length,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,3600.0,55000.0,5.91,675.0,679.0,13.99,123.03,7.0,13.0,2765.0,0.0,30.0,0.0,0.0,148.0,0,1,2,13,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,24700.0,65000.0,16.06,715.0,719.0,11.99,820.28,22.0,38.0,21470.0,1.0,6.0,0.0,0.0,192.0,0,1,2,10,11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,1.0,12.0,0.0,0.0,210.0,1,1,5,25,4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,11950.0,34000.0,10.20,690.0,694.0,13.44,405.18,5.0,6.0,8822.0,0.0,0.0,0.0,0.0,338.0,0,1,2,12,5,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,20000.0,180000.0,14.67,680.0,684.0,9.17,637.58,12.0,27.0,87329.0,0.0,49.0,0.0,0.0,306.0,0,0,1,6,11,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319505,18000.0,130000.0,20.59,735.0,739.0,9.49,377.95,17.0,39.0,23833.0,0.0,0.0,0.0,0.0,147.0,1,0,1,6,6,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1319506,29400.0,180792.0,22.03,705.0,709.0,13.99,683.94,16.0,32.0,77480.0,0.0,0.0,0.0,0.0,175.0,1,0,2,12,10,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1319507,32000.0,157000.0,10.34,735.0,739.0,14.49,752.74,14.0,18.0,111598.0,0.0,0.0,0.0,0.0,64.0,1,0,2,13,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1319508,16000.0,150000.0,12.25,665.0,669.0,12.79,362.34,12.0,28.0,7700.0,0.0,68.0,0.0,4.0,230.0,1,0,2,10,11,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [114]:
y.reset_index(drop=True, inplace=True)
y

0          1.0
1          1.0
2          1.0
3          1.0
4          1.0
          ... 
1319505    1.0
1319506    1.0
1319507    0.0
1319508    1.0
1319509    0.0
Name: loan_status_bin, Length: 1319510, dtype: float64

In [115]:
X_scaled = sc.fit_transform(X)

In [116]:
X = pd.DataFrame(X_scaled, columns=X.columns)
X

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,months_cr_line,term,initial_list_status,grade,sub_grade,emp_length,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,-1.239239,-0.307239,-1.446628,-0.663160,-0.663151,0.163792,-1.203010,-0.840635,-1.000430,-0.603176,-0.362263,0.563076,-0.065598,-0.356859,-0.522937,-0.560026,0.853561,0.199479,0.361317,1.124204,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,0.852241,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,1.523538,-0.798653,-0.669432
1,1.197526,-0.165064,-0.233691,0.598399,0.598384,-0.257756,1.480721,1.899426,1.082545,0.232773,0.776085,-0.477117,-0.065598,-0.356859,-0.034138,-0.560026,0.853561,0.199479,-0.104144,1.124204,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,-1.173377,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,9.269533,-0.082564,-0.041732,1.523538,-0.798653,-0.669432
2,-0.453931,0.395575,0.878865,-0.032380,-0.032384,1.946941,-0.560685,0.072718,0.832588,0.253287,0.776085,-0.217069,-0.065598,-0.356859,0.165825,1.785632,0.853561,2.517044,2.223161,-0.660074,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,-1.173377,-0.01572,-0.263150,-0.073474,6.678987,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,-0.656367,1.252108,-0.669432
3,-0.274927,-0.605807,-0.933968,-0.190075,-0.190075,0.047867,-0.117009,-1.205977,-1.583663,-0.332481,-0.362263,-0.737165,-0.065598,-0.356859,1.587785,-0.560026,0.853561,0.199479,0.206163,-0.405177,-0.014724,-0.982498,-0.005904,-0.010447,-0.347838,1.223383,-0.104967,-0.531727,0.852241,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,-0.656367,1.252108,-0.669432
4,0.654739,1.469949,-0.399798,-0.505465,-0.505459,-0.852138,0.777504,0.072718,0.166036,3.176092,-0.362263,1.386562,-0.065598,-0.356859,1.232295,-0.560026,-1.171562,-0.573042,-0.724759,1.124204,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,0.852241,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,1.523538,-0.798653,-0.669432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319505,0.423766,0.759073,0.307649,1.229179,1.229151,-0.784691,-0.221817,0.986072,1.165864,0.338379,-0.362263,-0.737165,-0.065598,-0.356859,-0.534046,1.785632,-1.171562,-0.573042,-0.724759,-0.150280,-0.014724,-0.982498,-0.005904,-0.010447,2.874900,-0.817406,-0.104967,-0.531727,-1.173377,-0.01572,3.800107,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,1.523538,-0.798653,-0.669432
1319506,1.740312,1.481209,0.479731,0.283010,0.283000,0.163792,0.955945,0.803401,0.582631,2.735928,-0.362263,-0.737165,-0.065598,-0.356859,-0.222992,1.785632,-1.171562,0.199479,0.206163,0.869308,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,0.852241,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,1.523538,-0.798653,-0.669432
1319507,2.040577,1.142946,-0.917238,1.229179,1.229151,0.269179,1.220758,0.438060,-0.583835,4.260703,-0.362263,-0.737165,-0.065598,-0.356859,-1.456099,1.785632,-1.171562,0.199479,0.361317,-0.660074,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,-1.173377,-0.01572,3.800107,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,-0.656367,1.252108,-0.669432
1319508,0.192794,

In [117]:
scaled = pd.concat([X,y], axis=1)
scaled

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,pub_rec,months_cr_line,term,initial_list_status,grade,sub_grade,emp_length,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified,loan_status_bin
0,-1.239239,-0.307239,-1.446628,-0.663160,-0.663151,0.163792,-1.203010,-0.840635,-1.000430,-0.603176,-0.362263,0.563076,-0.065598,-0.356859,-0.522937,-0.560026,0.853561,0.199479,0.361317,1.124204,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,0.852241,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,1.523538,-0.798653,-0.669432,1.0
1,1.197526,-0.165064,-0.233691,0.598399,0.598384,-0.257756,1.480721,1.899426,1.082545,0.232773,0.776085,-0.477117,-0.065598,-0.356859,-0.034138,-0.560026,0.853561,0.199479,-0.104144,1.124204,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,-1.173377,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,9.269533,-0.082564,-0.041732,1.523538,-0.798653,-0.669432,1.0
2,-0.453931,0.395575,0.878865,-0.032380,-0.032384,1.946941,-0.560685,0.072718,0.832588,0.253287,0.776085,-0.217069,-0.065598,-0.356859,0.165825,1.785632,0.853561,2.517044,2.223161,-0.660074,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,-1.173377,-0.01572,-0.263150,-0.073474,6.678987,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,-0.656367,1.252108,-0.669432,1.0
3,-0.274927,-0.605807,-0.933968,-0.190075,-0.190075,0.047867,-0.117009,-1.205977,-1.583663,-0.332481,-0.362263,-0.737165,-0.065598,-0.356859,1.587785,-0.560026,0.853561,0.199479,0.206163,-0.405177,-0.014724,-0.982498,-0.005904,-0.010447,-0.347838,1.223383,-0.104967,-0.531727,0.852241,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,-0.656367,1.252108,-0.669432,1.0
4,0.654739,1.469949,-0.399798,-0.505465,-0.505459,-0.852138,0.777504,0.072718,0.166036,3.176092,-0.362263,1.386562,-0.065598,-0.356859,1.232295,-0.560026,-1.171562,-0.573042,-0.724759,1.124204,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,0.852241,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,1.523538,-0.798653,-0.669432,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319505,0.423766,0.759073,0.307649,1.229179,1.229151,-0.784691,-0.221817,0.986072,1.165864,0.338379,-0.362263,-0.737165,-0.065598,-0.356859,-0.534046,1.785632,-1.171562,-0.573042,-0.724759,-0.150280,-0.014724,-0.982498,-0.005904,-0.010447,2.874900,-0.817406,-0.104967,-0.531727,-1.173377,-0.01572,3.800107,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,1.523538,-0.798653,-0.669432,1.0
1319506,1.740312,1.481209,0.479731,0.283010,0.283000,0.163792,0.955945,0.803401,0.582631,2.735928,-0.362263,-0.737165,-0.065598,-0.356859,-0.222992,1.785632,-1.171562,0.199479,0.206163,0.869308,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,0.852241,-0.01572,-0.263150,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.041732,1.523538,-0.798653,-0.669432,1.0
1319507,2.040577,1.142946,-0.917238,1.229179,1.229151,0.269179,1.220758,0.438060,-0.583835,4.260703,-0.362263,-0.737165,-0.065598,-0.356859,-1.456099,1.785632,-1.171562,0.199479,0.361317,-0.660074,-0.014724,1.017814,-0.005904,-0.010447,-0.347838,-0.817406,-0.104967,-0.531727,-1.173377,-0.01572,3.800107,-0.073474,-0.149723,-0.107582,-0.0842,-0.247959,-0.026386,-0.107880,-0.082564,-0.04173

In [118]:
scaled.to_csv('All_Accepted_44.csv', index=False)